In [148]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import torchio as tio
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchviz
import graphviz


from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


from functions import *

In [149]:
metaData = pd.read_csv('../metaDataTrain.csv')
metaDataClean = metaData.iloc
subject_list = load_training_dataset(metaDataClean)

## Let us construct the features vectors

In [150]:
features = np.zeros((100, 14))  ## there are 5 features and 20 subjects per training class

features_metadata = select_row_x_and_y_from_table(metaDataClean, 2, 3)

## adding into features vectors
features[:,:2] = features_metadata

Let us add volume information to feature vectors

In [151]:
volumes_0_ed = []
volumes_1_ed = []
volumes_2_ed = []
volumes_3_ed = []

volumes_0_es = []
volumes_1_es = []
volumes_2_es = []
volumes_3_es = []
   
for i in range(0, 100):
    subject = subject_list[i]
    
    ed_view = subject.ed_seg
    volumes_0_ed.append(get_volume(ed_view, 0))
    volumes_1_ed.append(get_volume(ed_view, 1))
    volumes_2_ed.append(get_volume(ed_view, 2))
    volumes_3_ed.append(get_volume(ed_view, 3))

    es_view = subject.es_seg
    volumes_0_es.append(get_volume(es_view, 0))
    volumes_1_es.append(get_volume(es_view, 1))
    volumes_2_es.append(get_volume(es_view, 2))
    volumes_3_es.append(get_volume(es_view, 3))

volumes_0_ed = np.array([volumes_0_ed]).flatten()
volumes_1_ed = np.array([volumes_1_ed]).flatten() 
volumes_2_ed = np.array([volumes_2_ed]).flatten() 
volumes_3_ed = np.array([volumes_3_ed]).flatten() 

volumes_0_es = np.array([volumes_0_es]).flatten()
volumes_1_es = np.array([volumes_1_es]).flatten() 
volumes_2_es = np.array([volumes_2_es]).flatten() 
volumes_3_es = np.array([volumes_3_es]).flatten() 

######### new features
ejection_fraction_right = (volumes_1_ed - volumes_1_es) / volumes_1_ed
ejection_fraction_left = (volumes_3_ed - volumes_3_es) / volumes_3_ed

volumes_0_ed = np.array([volumes_0_ed]).flatten()
volumes_1_ed = np.array([volumes_1_ed]).flatten() / volumes_0_ed
volumes_2_ed = np.array([volumes_2_ed]).flatten() / volumes_0_ed
volumes_3_ed = np.array([volumes_3_ed]).flatten() / volumes_0_ed

volumes_0_es = np.array([volumes_0_es]).flatten()
volumes_1_es = np.array([volumes_1_es]).flatten() / volumes_0_es
volumes_2_es = np.array([volumes_2_es]).flatten() / volumes_0_es
volumes_3_es = np.array([volumes_3_es]).flatten() / volumes_0_es

ratioRL_ed = volumes_3_ed / volumes_1_ed
ratioRL_es = volumes_3_es / volumes_1_es

ratioML_ed = volumes_2_ed / volumes_1_ed
ratioML_es = volumes_2_es / volumes_1_es


"""for i, new_col in enumerate([ejection_fraction_right[:], volumes_1_ed[:], volumes_2_ed[:], volumes_3_ed[:], ejection_fraction_left[:], volumes_1_es[:], volumes_2_es[:], volumes_3_es[:]]):
    features[:,i+2] = new_col"""

for i, new_col in enumerate([ratioRL_ed[:], ratioRL_es[:], ratioML_ed[:], ratioML_es[:], ejection_fraction_right[:], volumes_1_ed[:], volumes_2_ed[:], volumes_3_ed[:], ejection_fraction_left[:], volumes_1_es[:], volumes_2_es[:], volumes_3_es[:]]):
    features[:,i+2] = new_col

    

NiftiImageIO (0x558097eee3c0): ../Train/001/001_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/001/001_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/001/001_ES_seg.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/001/001_ES_seg.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/002/002_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/002/002_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/002/002_ES_seg.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/002/002_ES_seg.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/003/003_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/003/003_ED_seg.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/003/003_ES_seg.nii has unexpected scales in sform


Let us just give our explicit class vector

In [152]:
classes = np.zeros((100))
classes[:20] = 2
classes[20:40] = 3
classes[40:60] = 1
classes[60:80] = 0
classes[80:100] = 4
classes = np.array(classes.astype('int'))

In [153]:
shapes = np.array([subject.ed.shape for subject in subject_list])
max1, max2, max3 = shapes[:,1].max(), shapes[:,2].max(), shapes[:,3].max()
shaper = tio.CropOrPad((max1, max2, max3))

X_train_ed = np.array([shaper(subject.ed_seg) for subject in subject_list])

NiftiImageIO (0x558097eee3c0): ../Train/001/001_ED.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/002/002_ED.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/003/003_ED.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/004/004_ED.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/005/005_ED.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/006/006_ED.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/007/007_ED.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/008/008_ED.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/009/009_ED.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/010/010_ED.nii has unexpected scales in sform

NiftiImageIO (0x558097eee3c0): ../Train/011/011_ED.nii has unexpected scales in sform

NiftiImageIO (0x55809e1936c0): ../Train/012

In [154]:
class aTokenToRullThemAll(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv3d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv4 = nn.Conv3d(32, 32, 3, padding=1)
        self.conv5 = nn.Conv3d(32, 16, 3, padding=1)
        self.conv6 = nn.Conv3d(16, 8, 3, padding=1)

        self.maxpool2 = nn.MaxPool3d(2, 2)
        
        self.denseim1 = nn.Linear(int(6656), 128)
        self.denseim2 = nn.Linear(128, 16)

        self.denseclass1 = nn.Linear(30, 12)
        self.denseclass2 = nn.Linear(12, 5)

        self.batchnorm16 = nn.BatchNorm3d(16)
        self.batchnorm32 = nn.BatchNorm3d(32)
        self.batchnorm8 = nn.BatchNorm3d(8)

    def forward(self, im, stats, testing_vector, batchsize):

        image_features = F.leaky_relu(self.conv1(im))
        image_features = self.batchnorm16(image_features)

        image_features = F.leaky_relu(self.conv2(image_features))
        image_features = self.maxpool2(image_features)
        image_features = self.batchnorm32(image_features)

        image_features = F.leaky_relu(self.conv4(image_features))
        image_features = self.maxpool2(image_features)
        image_features = self.batchnorm32(image_features)
    

        image_features = F.leaky_relu(self.conv5(image_features))
        image_features = self.maxpool2(image_features)
        image_features = self.batchnorm16(image_features)


        image_features = F.leaky_relu(self.conv6(image_features))
        image_features = self.maxpool2(image_features)
        image_features = self.batchnorm8(image_features)


        image_features = F.leaky_relu(self.denseim1(image_features.view(-1, 6656)))
        image_features = F.leaky_relu(self.denseim2(image_features))

        features = torch.cat((image_features, stats), dim = 1)
        
        features *= testing_vector ### FOR DEBUGGING PURPOSES

        features = F.leaky_relu(self.denseclass1(features))

        classification = F.softmax(self.denseclass2(features), dim = 1)

        return classification
    
class customds(Dataset):
    def __init__(self, images, features, targets):
        self.images = images
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        image = self.images[index]
        feature = self.features[index]
        target = self.targets[index]
        return image, feature, target

In [155]:
input = X_train_ed[2].data.float().requires_grad_()
input.shape


torch.Size([1, 428, 512, 18])

In [156]:
model_viz = aTokenToRullThemAll()
input = X_train_ed[2].data.float().requires_grad_()
testing_vector = torch.cat((torch.zeros((1, 16)), torch.ones((1, 14))), dim = 1)
features = torch.from_numpy(features).float()

dot = torchviz.make_dot_from_trace(model_viz(torch.unsqueeze(input, dim = 0), torch.unsqueeze(features[2], dim = 0), testing_vector, 1))

dot.format = 'svg'
dot.render('model', directory='./', cleanup=True)


NotImplementedError: This function has been moved to pytorch core and can be found here: https://pytorch.org/docs/stable/tensorboard.html

In [8]:
torch.cuda.empty_cache()
model = encoder()
criterion = torch.nn.CrossEntropyLoss()
lr = 0.005
batchsize = 12
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [9]:
images = torch.stack([X_train_ed[i].data for i in range(100)]).float().requires_grad_()
im_mean = torch.mean(images)
im_std = torch.std(images)
images = (images-im_mean)/im_std

classes = torch.from_numpy(classes).float().requires_grad_()

scaler = StandardScaler()
feaures = scaler.fit_transform(features)
features = torch.from_numpy(features).float().requires_grad_()

dataset = customds(images, features, classes)

dataloader = DataLoader(dataset, batch_size=batchsize, shuffle=True, pin_memory=True)

In [10]:
def to_device(data, device):

    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
        
    return data.to(device, non_blocking = True)

class DeviceDataLoader():

    def __init__(self, dl, device):
        self.dl = dl 
        self.device = device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    
    def __len__(self):
        return len(self.dl)

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

device = torch.device('cuda:0')
model.to(device)
dataloader = DeviceDataLoader(dataloader, device)
testing_vector = torch.cat((torch.zeros((batchsize, 16)), torch.ones((batchsize, 14))), dim = 1).to(device)



device_ids = [0,1,2]
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")
    model = nn.DataParallel(model, device_ids=device_ids)

model.to(device)

Using 3 GPUs


/home/infres/egardes-21/.local/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:32: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


DataParallel(
  (module): encoder(
    (conv1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (conv2): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (conv4): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (conv5): Conv3d(32, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (conv6): Conv3d(16, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (maxpool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (denseim1): Linear(in_features=6656, out_features=128, bias=True)
    (denseim2): Linear(in_features=128, out_features=16, bias=True)
    (denseclass1): Linear(in_features=30, out_features=12, bias=True)
    (denseclass2): Linear(in_features=12, out_features=5, bias=True)
    (batchnorm16): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (batchnorm32): BatchNorm3d(32, eps=1e-05, momentum=0.1, 

In [11]:
def model_acc(predict,labels):
  accuracy = torch.sum(predict == labels).item() / len(labels)
  return accuracy

def vector_to_class(x):
  _, y = torch.max(x, dim = 1)
  return y

In [12]:
n_epochs = 10
train_losses=[]
valid_losses=[]
i = 0


for epoch in range(0,n_epochs):
  train_loss=0.0
  torch.cuda.empty_cache()
  for batch_images, batch_features, batch_targets in dataloader:

    
    optimizer.zero_grad()
    predict = model(batch_images, batch_features, testing_vector, batchsize)
    # apply loss function
    prediction = vector_to_class(predict).float()
    loss=criterion(prediction,batch_targets)
    # backpropagation
    loss.backward()
    optimizer.step()
    train_loss=loss.item()



  print('Epoch:{} Train Loss:{:.4f}'.format(epoch+1,train_loss/batch_images.shape[0]))
  # calculate accuracy

  print('Accuracy:{:.2f}'.format(model_acc(prediction,batch_targets)))
# END STUDENT CODE

OutOfMemoryError: Caught OutOfMemoryError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/home/infres/egardes-21/.local/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/infres/egardes-21/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_1756097/2432889775.py", line 26, in forward
    image_features = F.leaky_relu(self.conv1(im))
  File "/home/infres/egardes-21/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/infres/egardes-21/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py", line 613, in forward
    return self._conv_forward(input, self.weight, self.bias)
  File "/home/infres/egardes-21/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py", line 60